<a href="https://colab.research.google.com/github/vcorredorg/SenalesySistemas2025/blob/main/Taller2/Transformada_de_fourier/c_app_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APP comunicaciones- modulación AM y circuitos eléctricos

In [13]:
!pip install streamlit yt-dlp librosa soundfile matplotlib -q

In [14]:
!mkdir -p pages

##Página principal – Bienvenida

In [22]:
%%writefile 0_👋_Bienvenida.py
import streamlit as st

st.set_page_config(
    page_title="Laboratorio FFT, AM y THD",
    page_icon="👋",
)

st.write("#Ejercicios de FFT, AM y Calidad de Energía 👋")

st.sidebar.success("Usa la barra lateral para elegir la sección.")

st.markdown(
    r"""
En este dashboard se estudian dos aplicaciones clásicas de la **Transformada de Fourier (FFT)**:

---

## 1. Aplicación en comunicaciones – Modulación AM

- Se utiliza un fragmento real de audio (descargado de YouTube) como **señal mensaje** \(m(t)\).
- Se construye la señal **modulada en amplitud (AM)** con índice de modulación $\mu$.
- Se genera también la versión **DSB-SC** para usar un **demodulador síncrono** como el de la figura:
  - Mezclador con la portadora local.
  - Filtro pasa bajas ideal implementado en el dominio de la frecuencia (FFT).
  - Etapa de escala para recuperar \(m(t)\).
- Se muestran las señales:
  - en el **tiempo**,
  - en el **dominio de la frecuencia** (magnitud),
  - y se reproducen los **audios** en cada etapa.

---

## 2. Aplicación en circuitos eléctricos – THD y factor de potencia

- Se repasan los conceptos de:
  - **Distorsión Total de Armónicos (THD)** de corriente.
  - **Factor de distorsión** y **factor de potencia total**.
- Se simula un **rectificador de onda completa** con:
  1. Carga **netamente resistiva**.
  2. Carga **R–C en serie**.
- A partir de la FFT de la corriente, se calcula:
  $$
  \mathrm{THD}_I = \frac{\sqrt{\sum_{k=2}^{\infty} I_k^2}}{I_1},
  \qquad
  \mathrm{DF} = \frac{1}{\sqrt{1 + \mathrm{THD}_I^2}},
  \qquad
  \mathrm{FP} = \cos(\varphi_1)\,\mathrm{DF}.
  $$
- Se comparan los resultados para distintas R y C.

---

Usa el menú de la izquierda para explorar:

1. 📡 Modulación y demodulación AM
2. ⚡ THD y factor de potencia en rectificador

Experimenta con los parámetros y observa cómo cambian las formas de onda, espectros, THD y FP.
"""
)


Overwriting 0_👋_Bienvenida.py


##Página 1 – Modulación / Demodulación AM (con yt-dlp + cookies opcional)

In [16]:
%%writefile pages/1_📡_AM_Modulacion_Demodulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

import librosa
import soundfile as sf
import yt_dlp
import io
import tempfile
import os

st.markdown("# 📡 Aplicación en comunicaciones: Modulación y demodulación AM")
st.sidebar.header("Sección 1: Modulación AM")

st.markdown(
    r"""
Usaremos un fragmento de audio real (YouTube) como mensaje \(m(t)\)
para estudiar modulación AM y demodulación síncrona.

- Portadora: \(c(t) = A_c \cos(2\pi f_c t)\)
- AM con portadora:
  $$
  y_{\text{AM}}(t) = A_c(1 + \mu m(t))\cos(2\pi f_c t)
  $$
- DSB-SC (para el demodulador de la figura):
  $$
  y_{\text{DSB-SC}}(t) = A_1 m(t)\cos(2\pi f_c t)
  $$

El demodulador síncrono implementa:
1. Mezclador con la portadora local.
2. Filtro pasa bajas ideal (en frecuencia, usando FFT).
3. Ganancia $2/A_1$ para recuperar m(t).
"""
)

# Funciones auxiliares
def cargar_audio_youtube(url, inicio_s, duracion_s, fs_objetivo, cookies_file=None):
    """
    Descarga audio de YouTube usando yt-dlp, extrae un fragmento
    y lo remuestra a fs_objetivo.

    Si cookies_file no es None, se guarda como cookies.txt temporal
    y se pasa a yt-dlp (útil para videos restringidos).
    """
    tmp_dir = tempfile.mkdtemp()

    cookie_path = None
    if cookies_file is not None:
        cookie_path = os.path.join(tmp_dir, "cookies.txt")
        with open(cookie_path, "wb") as f:
            f.write(cookies_file.getvalue())

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(tmp_dir, "audio.%(ext)s"),
        "quiet": True,
        "noplaylist": True,
    }
    if cookie_path is not None:
        ydl_opts["cookiefile"] = cookie_path

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)

    audio, sr = librosa.load(
        filename,
        sr=fs_objetivo,
        offset=float(inicio_s),
        duration=float(duracion_s),
        mono=True,
    )
    audio = audio.astype(np.float32)
    return audio, fs_objetivo


def a_bytes_audio(x, fs):
    """Convierte un array numpy en bytes WAV para st.audio."""
    buf = io.BytesIO()
    sf.write(buf, x, fs, format="WAV")
    return buf.getvalue()


def graficar_tiempo_y_espectro(x, fs, titulo, t_max=0.02):
    """Grafica señal en tiempo (ventana corta) y espectro de magnitud."""
    N = len(x)
    t = np.arange(N) / fs

    fig, axs = plt.subplots(1, 2, figsize=(10, 3))

    # Tiempo
    idx = t <= min(t_max, t[-1])
    axs[0].plot(t[idx], x[idx])
    axs[0].set_xlabel("Tiempo [s]")
    axs[0].set_title(titulo + " (tiempo)")

    # Espectro
    ventana = np.hanning(N)
    X = np.fft.rfft(x * ventana)
    f = np.fft.rfftfreq(N, 1 / fs)
    axs[1].plot(f, np.abs(X))
    axs[1].set_xlabel("Frecuencia [Hz]")
    axs[1].set_ylabel("|X(f)|")
    axs[1].set_title(titulo + " (espectro)")
    axs[1].set_xlim(0, fs / 2)

    st.pyplot(fig)


def filtro_pasabajas_ideal_fft(x, fs, fcorte_hz):
    """Filtrado pasa bajas ideal implementado en frecuencia con FFT."""
    N = len(x)
    X = np.fft.rfft(x)
    f = np.fft.rfftfreq(N, 1 / fs)
    X[f > fcorte_hz] = 0.0
    y = np.fft.irfft(X, n=N)
    return y


# Controles de la barra lateral-
st.sidebar.subheader("Parámetros de simulación")

url = st.sidebar.text_input(
    "URL de YouTube (audio)",
    value="https://www.youtube.com/watch?v=dQw4w9WgXcQ",
)

inicio_seg = st.sidebar.number_input(
    "Segundo inicial del fragmento", min_value=0.0, value=20.0, step=1.0
)
duracion_seg = st.sidebar.number_input(
    "Duración del fragmento [s]", min_value=1.0, max_value=10.0, value=5.0, step=1.0
)

fs_obj = st.sidebar.number_input(
    "Frecuencia de muestreo de trabajo [Hz]",
    min_value=8000,
    max_value=48000,
    value=44100,
    step=1000,
)

fc = st.sidebar.slider(
    "Frecuencia de la portadora fc [Hz]",
    min_value=500,
    max_value=10000,
    value=4000,
    step=100,
)

mu = st.sidebar.slider(
    "Índice de modulación μ", min_value=0.1, max_value=1.5, value=1.0, step=0.1
)

fcorte = st.sidebar.slider(
    "Frecuencia de corte LPF ideal [Hz]",
    min_value=500,
    max_value=10000,
    value=5000,
    step=100,
)

cookies_file = st.sidebar.file_uploader(
    "cookies.txt de YouTube (opcional)", type=["txt"],
    help="Para videos restringidos, exporta tus cookies de YouTube como cookies.txt y súbelas aquí.",
)

st.sidebar.markdown("---")
procesar = st.sidebar.button("Descargar audio y simular AM + demodulación")


#Lógica principal
if procesar:
    try:
        st.info("Descargando audio de YouTube y procesando…")

        m_raw, fs = cargar_audio_youtube(
            url, inicio_seg, duracion_seg, fs_obj, cookies_file=cookies_file
        )

        # Normalizar mensaje a [-1, 1]
        if np.max(np.abs(m_raw)) > 0:
            m = m_raw / np.max(np.abs(m_raw))
        else:
            m = m_raw

        N = len(m)
        t = np.arange(N) / fs

        # Portadora
        Ac = 1.0
        c = Ac * np.cos(2 * np.pi * fc * t)

        # AM con portadora (AM-DSB-LC)
        y_am = Ac * (1 + mu * m) * np.cos(2 * np.pi * fc * t)

        # DSB-SC para demodulador síncrono
        A1 = 1.0
        y_dsb = A1 * m * np.cos(2 * np.pi * fc * t)

        # Mezclador (θ0 = 0)
        lo = np.cos(2 * np.pi * fc * t)
        y_mix = y_dsb * lo

        # LPF ideal
        y_lpf = filtro_pasabajas_ideal_fft(y_mix, fs, fcorte)

        # Escala final (2/A1)
        m_rec = 2 * y_lpf

        st.success("Simulación completada.")

        # ---------- Mensaje ----------
        st.subheader("Mensaje \(m(t)\) – fragmento de la canción")
        graficar_tiempo_y_espectro(m, fs, "Mensaje m(t)")
        st.audio(a_bytes_audio(m, fs), format="audio/wav")

        # ---------- Portadora ----------
        st.subheader("Portadora \(c(t)\)")
        graficar_tiempo_y_espectro(c, fs, "Portadora c(t)")
        st.audio(a_bytes_audio(c, fs), format="audio/wav")

        # ---------- AM con portadora ----------
        st.subheader("Señal AM con portadora")
        graficar_tiempo_y_espectro(y_am, fs, "Señal AM (con portadora)")
        st.audio(a_bytes_audio(y_am, fs), format="audio/wav")

        # ---------- DSB-SC ----------
        st.subheader("Señal DSB-SC (portadora suprimida)")
        graficar_tiempo_y_espectro(y_dsb, fs, "Señal DSB-SC")
        st.audio(a_bytes_audio(y_dsb, fs), format="audio/wav")

        # ---------- Mezclador ----------
        st.subheader("Salida del mezclador")
        graficar_tiempo_y_espectro(y_mix, fs, "Salida del mezclador")
        st.audio(a_bytes_audio(y_mix, fs), format="audio/wav")

        # ---------- LPF ----------
        st.subheader("Salida del filtro pasa bajas ideal")
        graficar_tiempo_y_espectro(y_lpf, fs, "Salida LPF (≈ A₁/2 · m(t))")
        st.audio(a_bytes_audio(y_lpf, fs), format="audio/wav")

        # ---------- Señal recuperada ----------
        st.subheader("Mensaje recuperado \(m_{\mathrm{rec}}(t)\)")
        graficar_tiempo_y_espectro(m_rec, fs, "Mensaje recuperado")
        st.audio(a_bytes_audio(m_rec, fs), format="audio/wav")

    except Exception as e:
        st.error(f"Ocurrió un problema descargando o procesando el audio: {e}")
else:
    st.info(
        "Introduce una URL de YouTube, (opcionalmente sube un cookies.txt), "
        "ajusta los parámetros en la barra lateral y pulsa "
        "**'Descargar audio y simular AM + demodulación'**."
    )


Overwriting pages/1_📡_AM_Modulacion_Demodulacion.py


##Página 2 – THD y Factor de Potencia en rectificador

In [17]:
%%writefile pages/2_⚡_THD_y_FP_Rectificador.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

st.markdown("# ⚡ Aplicación en circuitos eléctricos: THD y factor de potencia")

st.markdown(
    r"""
Analizaremos la **distorsión total de armónicos (THD)** y el **factor de potencia (FP)**
en un **rectificador de onda completa**, para dos tipos de carga:

1. Carga **netamente resistiva**.
2. Carga **R–C en serie**.

### Recordatorio teórico

Para una corriente periódica con armónicos $I_1, I_2, I_3, \dots$:

$$
\mathrm{THD}_I
= \frac{\sqrt{\sum_{k=2}^{\infty} I_k^2}}{I_1}.
$$

El **factor de distorsión** es

$$
\mathrm{DF} = \frac{1}{\sqrt{1 + \mathrm{THD}_I^2}},
$$

y el **factor de potencia total**:

$$
\mathrm{FP} = \cos(\varphi_1)\,\mathrm{DF},
$$

donde $\varphi_1$ es el ángulo entre la tensión fundamental y la corriente fundamental
(**factor de potencia de desplazamiento**).
"""
)

#Funciones auxiliares
def simular_rectificador_onda_completa(V_rms, f_red, R_R, R_RC, C_RC,
                                       n_ciclos, muestras_por_ciclo):
    """
    Simula un rectificador de onda completa alimentado por una fuente sinusoidal.
    - V_rms: tensión RMS de entrada
    - f_red: frecuencia de la red [Hz]
    - R_R: carga puramente resistiva
    - R_RC, C_RC: carga R–C en serie
    """
    w = 2 * np.pi * f_red
    Vm = np.sqrt(2) * V_rms

    fs = f_red * muestras_por_ciclo
    dt = 1.0 / fs
    N_total = int(n_ciclos * muestras_por_ciclo)
    t = np.arange(N_total) * dt

    # Tensión sinusoidal de entrada
    v_in = Vm * np.sin(w * t)

    # Salida del rectificador de onda completa: valor absoluto
    v_rect = np.abs(v_in)

    # Carga puramente resistiva
    i_R = v_rect / R_R

    # Carga R–C en serie
    # v_rect(t) = R_RC * i(t) + v_C(t), con i = C dvC/dt.
    vC = np.zeros_like(v_rect)
    i_RC = np.zeros_like(v_rect)
    alpha = dt / (R_RC * C_RC)

    for n in range(1, len(v_rect)):
        vC[n] = (vC[n - 1] + alpha * v_rect[n]) / (1 + alpha)
        i_RC[n] = C_RC * (vC[n] - vC[n - 1]) / dt

    return t, v_rect, i_R, i_RC, fs


def calcular_thd(signal, fs, f0, n_harm=20):
    """
    Calcula THD a partir de la FFT de 'signal'.
    - fs: frecuencia de muestreo
    - f0: frecuencia fundamental (aquí la de la onda rectificada)
    """
    N = len(signal)
    x = signal - np.mean(signal)
    ventana = np.hanning(N)
    X = np.fft.rfft(x * ventana)
    f = np.fft.rfftfreq(N, 1 / fs)
    mag = np.abs(X)

    # Fundamental
    idx1 = np.argmin(np.abs(f - f0))
    I1 = mag[idx1]

    suma_cuad = 0.0
    for k in range(2, n_harm + 1):
        fk = k * f0
        idxk = np.argmin(np.abs(f - fk))
        if idxk < len(mag):
            suma_cuad += mag[idxk] ** 2

    THD = np.sqrt(suma_cuad) / I1
    return THD


def graficar_corriente_y_espectro(t, i, fs, f_linea,
                                  titulo, n_ciclos_mostrar=4, fmax=None):
    """Muestra la corriente en tiempo (últimos ciclos) y su espectro de magnitud."""
    N = len(i)
    periodo = 1.0 / f_linea
    muestras_por_periodo = int(fs / f_linea)

    # Tomamos los últimos n_ciclos_mostrar ciclos
    N_mostrar = n_ciclos_mostrar * muestras_por_periodo
    if N_mostrar > N:
        N_mostrar = N
    i_seg = i[-N_mostrar:]
    t_seg = t[-N_mostrar:]

    fig, axs = plt.subplots(1, 2, figsize=(10, 3))

    # Tiempo
    axs[0].plot(t_seg, i_seg)
    axs[0].set_xlabel("Tiempo [s]")
    axs[0].set_title(titulo + " (tiempo)")

    # Espectro
    ventana = np.hanning(len(i_seg))
    X = np.fft.rfft(i_seg * ventana)
    f = np.fft.rfftfreq(len(i_seg), 1 / fs)
    axs[1].plot(f, np.abs(X))
    axs[1].set_xlabel("Frecuencia [Hz]")
    axs[1].set_ylabel("|I(f)|")
    axs[1].set_title(titulo + " (espectro)")
    if fmax is not None:
        axs[1].set_xlim(0, fmax)

    st.pyplot(fig)


#Controles de la barra lateral
st.sidebar.header("Parámetros del rectificador")

V_rms = st.sidebar.number_input("Tensión RMS de entrada [V]",
                                value=120.0, step=10.0)
f_linea = st.sidebar.number_input("Frecuencia de la red [Hz]",
                                  value=60.0, step=10.0)

R_R = st.sidebar.number_input("Carga resistiva R [Ω]",
                              value=100.0, step=10.0)
R_RC = st.sidebar.number_input("Resistencia en la carga R–C [Ω]",
                               value=100.0, step=10.0)
C_uF = st.sidebar.number_input("Capacidad en la carga R–C [µF]",
                               value=470.0, step=10.0)
C_RC = C_uF * 1e-6  # en faradios

n_ciclos = st.sidebar.slider("Número de ciclos simulados",
                             min_value=5, max_value=30, value=12)
muestras_por_ciclo = st.sidebar.slider(
    "Muestras por ciclo",
    min_value=500,
    max_value=5000,
    value=2000,
    step=500,
)

simular = st.sidebar.button("Simular rectificador, THD y FP")

#Lógica principal
if simular:
    t, v_rect, i_R, i_RC, fs = simular_rectificador_onda_completa(
        V_rms, f_linea, R_R, R_RC, C_RC, n_ciclos, muestras_por_ciclo
    )

    # Fundamental de la onda rectificada: 2*f_linea
    f0 = 2 * f_linea

    N_total = len(t)
    muestras_por_periodo_0 = int(fs / f0)
    N_ultimos = 6 * muestras_por_periodo_0
    if N_ultimos > N_total:
        N_ultimos = N_total

    i_R_steady = i_R[-N_ultimos:]
    i_RC_steady = i_RC[-N_ultimos:]

    # THD
    THD_R = calcular_thd(i_R_steady, fs, f0)
    THD_RC = calcular_thd(i_RC_steady, fs, f0)

    # Factor de distorsión
    DF_R = 1.0 / np.sqrt(1.0 + THD_R**2)
    DF_RC = 1.0 / np.sqrt(1.0 + THD_RC**2)

    # cos(phi1) fundamental de la red
    w1 = 2 * np.pi * f_linea
    # Carga resistiva: cosφ = 1
    cos_phi_R = 1.0
    # Carga R–C en serie: cosφ = R/|Z|
    Xc1 = 1.0 / (w1 * C_RC)
    cos_phi_RC = R_RC / np.sqrt(R_RC**2 + Xc1**2)

    FP_R = cos_phi_R * DF_R
    FP_RC = cos_phi_RC * DF_RC

    #Resultados numéricos
    st.subheader("Resultados numéricos")

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("### Carga puramente resistiva")
        st.write(f"THD de corriente = {THD_R*100:.2f} %")
        st.write(f"Factor de distorsión DF = {DF_R:.4f}")
        st.write(f"cos(φ₁) (desplazamiento) = {cos_phi_R:.4f}")
        st.write(f"Factor de potencia total FP = {FP_R:.4f}")

    with col2:
        st.markdown("### Carga R–C en serie")
        st.write(f"THD de corriente = {THD_RC*100:.2f} %")
        st.write(f"Factor de distorsión DF = {DF_RC:.4f}")
        st.write(f"cos(φ₁) (desplazamiento) = {cos_phi_RC:.4f}")
        st.write(f"Factor de potencia total FP = {FP_RC:.4f}")

    st.markdown(
        r"""
**Comentario:**

- La carga **resistiva** tiene $\cos\varphi_1 \approx 1$ y menor distorsión → FP alto.
- La carga **R–C** introduce desfase y corrientes más pulsantes:
  - disminuye $\cos\varphi_1$,
  - aumenta la THD,
  - empeora el factor de potencia total.
"""
    )

    #Gráficas
    st.subheader("Formas de onda y espectros de corriente")

    st.markdown("#### Carga puramente resistiva")
    graficar_corriente_y_espectro(
        t, i_R, fs, f_linea,
        "Corriente carga resistiva",
        n_ciclos_mostrar=4,
        fmax=10 * f0,
    )

    st.markdown("#### Carga R–C en serie")
    graficar_corriente_y_espectro(
        t, i_RC, fs, f_linea,
        "Corriente carga R–C",
        n_ciclos_mostrar=4,
        fmax=10 * f0,
    )
else:
    st.info("Configura los parámetros en la barra lateral y pulsa **'Simular rectificador, THD y FP'**.")


Overwriting pages/2_⚡_THD_y_FP_Rectificador.py


##Instalar cloudflared y lanzar *Streamlit*

In [18]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared


--2025-12-05 15:52:47--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-05 15:52:48--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-05T16%3A38%3A50Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-05

##Ejecutar Streamlit + túnel

In [19]:
!mkdir -p /content/logs
!streamlit run 0_👋_Bienvenida.py &>/content/logs/streamlit.log &

!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &


##Leer la URL pública

In [20]:
import time, re

time.sleep(5)  # pequeño tiempo para que se genere la URL

public_url = None
with open('/content/cloudflared.log') as f:
    found_context = False
    for line in f:
        if "Your quick Tunnel has been created" in line:
            found_context = True
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                public_url = match.group(0)
                break

if public_url:
    print(f'Tu aplicación está disponible en: {public_url}')
else:
    print("No se pudo encontrar la URL pública. Revisa /content/cloudflared.log")


Tu aplicación está disponible en: https://mas-dakota-nuts-units.trycloudflare.com


##Celda para apagar Streamlit

In [21]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.strip() == "1":
    os.system("pkill streamlit")
    print("El proceso de Streamlit ha sido finalizado.")


Digite (1) para finalizar la ejecución del Dashboard: 0
